In [1]:
# !python -m pip uninstall tensorflow

In [2]:
import tensorflow as tf

2025-11-22 10:05:38.415402: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import pandas as pd

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
from string import punctuation

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import CategoryEncoding, Input, Dense, LSTM, StringLookup, Embedding

In [10]:
from sklearn.model_selection import train_test_split

In [25]:
print("Versão do Tensorflow: ", tf.__version__)

Versão do Tensorflow:  2.20.0


In [11]:
nltk.download('punkt') # Necessário para word_tokenize
nltk.download('stopwords')
nltk.download('punkt_tab')
palavras_remover = set(stopwords.words())

[nltk_data] Downloading package punkt to /home/anton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/anton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/anton/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
df = pd.read_csv("../../../dados/nlp/dialogs.txt", sep="\t" , encoding="utf-8", 
                 header=None, names=["Perguntas", "Respostas"])

In [13]:
df["Perguntas"].shape

(3725,)

In [14]:
entradas = df["Perguntas"] + df["Respostas"]
entradas.shape

(3725,)

In [15]:
def limpar_texto( texto ):
    translator = str.maketrans('—’', '  ', punctuation)
    texto_limpo = texto.lower().translate( translator )
    tokens = word_tokenize(texto_limpo)
    tokens_limpos = []
    for token in tokens:
        # if token not in palavras_remover:
        tokens_limpos.append(token)
    return " ".join(tokens_limpos)

In [16]:
entradas_limpo = entradas.apply( limpar_texto )

In [17]:
MAX_PALAVRAS = 5000

In [18]:
tokenizer = Tokenizer(num_words = MAX_PALAVRAS, oov_token="<UNKNOW>") # OOV = Out Of Vocabulary

In [19]:
tokenizer.fit_on_texts( entradas_limpo )
len(tokenizer.word_index)

5629

In [20]:
sequencias = tokenizer.texts_to_sequences( entradas_limpo )

In [21]:
sequencias[0:5]

[[1463, 48, 15, 2, 1464, 619, 48, 26, 808],
 [36, 619, 48, 26, 2347, 159, 41, 363, 19, 527],
 [36, 159, 41, 363, 19, 2348, 193, 29, 48, 12, 2, 86],
 [43, 193, 29, 48, 12, 2, 2349, 86, 96, 16, 26, 2],
 [119, 86, 96, 16, 26, 2350, 86, 41, 36, 14, 99, 68, 137]]

In [22]:
entradas_features = []
saidas_labels = []

for sequencia in sequencias:
# [ 2, 65, 1, 43, 29]     ==> len(sequencia)  => 5
    for i in range(2, len(sequencia)):
        features = sequencia[0: i]
        label = sequencia[i]
        entradas_features.append( features )
        saidas_labels.append( label )
len(entradas_features),    len(saidas_labels)

(36786, 36786)

In [23]:
sequencias_padded = np.array(pad_sequences( entradas_features, padding="pre" ))
sequencias_padded.shape

(36786, 30)

In [24]:
output_encoder = CategoryEncoding( num_tokens=MAX_PALAVRAS, output_mode="one_hot" )
saida_encoded = output_encoder( saidas_labels ).numpy()
saida_encoded.shape

W0000 00:00:1763816743.976732   10069 gpu_device.cc:2431] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
W0000 00:00:1763816743.991268   10069 gpu_device.cc:2431] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1763816744.341521   10069 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5133 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 5060, pci bus id: 0000:03:00.0, compute capability: 12.0
2025-11-22 10:05:44.901848: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleLoadData(&module, data)' failed with 'CUDA_ERROR_INVALID_PTX'

2025-11-22 10:05:44.901920: W tensorflow/compiler/mlir/tools/kernel_gen/tf_gpu_runtime_wrappers.cc:40] 'cuModuleGetFunction

InternalError: Exception encountered when calling CategoryEncoding.call().

[1m{{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:Cast] name: [0m

Arguments received by CategoryEncoding.call():
  • inputs=['15', '2', '1464', '619', '48', '26', '808', '48', '26', '2347', '159', '41', '363', '19', '527', '41', '363', '19', '2348', '193', '29', '48', '12', '2', '86', '29', '48', '12', '2', '2349', '86', '96', '16', '26', '2', '96', '16', '26', '2350', '86', '41', '36', '14', '99', '68', '137', '41', '36', '14', '99', '68', '2351', '99', '8', '2', '34', '5', '8', '2', '34', '950', '34', '5', '951', '5', '2352', '2', '18', '7', '59', '18', '7', '1465', '130', '25', '6', '49', '148', '2353', '25', '6', '49', '148', '2354', '689', '40', '99', '40', '2355', '2', '124', '83', '2356', '194'

In [ ]:
modelo = Sequential()
modelo.add(  Embedding( input_dim=(MAX_PALAVRAS + 1), output_dim=50, mask_zero=False )  )
modelo.add( LSTM(128) )
modelo.add( Dense( MAX_PALAVRAS, activation="softmax") )
modelo.summary()

In [ ]:
modelo.compile( optimizer="adam", loss="categorical_crossentropy", metrics=["Accuracy"])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split( sequencias_padded, saida_encoded, test_size=0.05, random_state=100 )
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [ ]:
resultado = modelo.fit( X_train, Y_train, epochs=35, batch_size=32 )

In [ ]:
modelo.evaluate( X_test, Y_test )

In [ ]:
acuracia = resultado.history["Accuracy"]
perda = resultado.history["loss"] / np.max(resultado.history["loss"]) 
plt.plot(acuracia, color="green")
plt.plot(perda, color="red")

In [ ]:
texto = "hi i am Antonio what is your favorite kind of music is she gave"

In [ ]:
texto_limpo = limpar_texto( texto )
texto_limpo

In [ ]:
sequencias = tokenizer.texts_to_sequences( [texto_limpo] )
texto_seq_padded = pad_sequences( sequencias, maxlen=sequencias_padded.shape[1], padding="pre")
predicted = modelo.predict( texto_seq_padded )
pos_palavra = np.argmax( predicted )
for item in tokenizer.word_index.items():
    if item[1] == pos_palavra:
        print(item[0])
        break